# Required libararies 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import ttest_ind

# Getting the Very first dataset on 7 Counties in SJV

# fetching SJV data on 4 pollutants 

- PM2.5
- PM10
- NO2
- Ozone


In [ ]:
import requests
import pandas as pd
import time  # To avoid hitting API rate limits

# API endpoint for county-level data
url = "https://aqs.epa.gov/data/api/dailyData/byCounty"

# API Credentials
email = "dksxowns69@gmail.com"
api_key = "aquabird37"

# Pollutant parameter codes
params_list = [#"61101", # Wind Speed_Simple
               #"61103", # Wind Speed_ Resultant (Vector-based)
               #"61104", # Wind direction
               #"62101", # Outdoor Temperature
               #"62201", # Relative Humidity
               #"64101", # Barometric Pressure
               #"42101", # Carbon monoxide
               #"63101", # Visibility
               #"63011", # Solar Radiation
               "88101", # PM2.5
               "85129", # PM10
               "42602", # NO2
               "44201", # Ozone

]

# Year range
years = range(1980, 2025)

# California state code
state_code = "06"

# San Joaquin Valley county codes
county_codes = {
    "San Joaquin": "077",
    "Stanislaus": "099",
    "Merced": "047",
    "Fresno": "019",
    "Kings": "031",
    "Tulare": "107",
    "Kern": "029"
}

# Empty list to store data
all_data = []

# Loop through pollutants, counties, and years
for param in params_list:
    for county_name, county_code in county_codes.items():
        for year in years:
            print(f"Fetching {param} for {county_name} ({county_code}) in {year}...")

            params = {
                "email": email,
                "key": api_key,
                "param": param,
                "bdate": f"{year}0101",
                "edate": f"{year}1231",
                "state": state_code,
                "county": county_code
            }

            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                if "Data" in data and data["Data"]:
                    all_data.extend(data["Data"])
                    print(f"Success: {len(data['Data'])} records added.")
                else:
                    print(f"No data found for {param} in {county_name} ({year}).")
            else:
                print(f"Error: {response.status_code}, {response.text}")

            time.sleep(1)  # Avoid exceeding API rate limits

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Save to CSV for future analysis
df.to_csv("SJV_AQI_1980_2025.csv", index=False)

print(f"Final dataset contains {df.shape[0]} records.")


Fetching 62201 for Fresno (019) in 1999...
Success: 1249 records added.
Fetching 62201 for Fresno (019) in 2000...
Success: 1151 records added.
Fetching 62201 for Fresno (019) in 2001...
Success: 1319 records added.
Fetching 62201 for Fresno (019) in 2002...
Success: 1386 records added.
Fetching 62201 for Fresno (019) in 2003...
Success: 1635 records added.
Fetching 62201 for Fresno (019) in 2004...
Success: 1260 records added.
Fetching 62201 for Fresno (019) in 2005...
Success: 1090 records added.
Fetching 62201 for Fresno (019) in 2006...
Success: 1071 records added.
Fetching 62201 for Fresno (019) in 2007...
Success: 1095 records added.
Fetching 62201 for Fresno (019) in 2008...
Success: 1090 records added.
Fetching 62201 for Fresno (019) in 2009...
Success: 1032 records added.
Fetching 62201 for Fresno (019) in 2010...
Success: 1076 records added.
Fetching 62201 for Fresno (019) in 2011...
Success: 1091 records added.
Fetching 62201 for Fresno (019) in 2012...
Success: 1067 records

# ML Analyses

## 1st SARIMA Attempt_Jun

In [ ]:
# --- PARAMETERS ---
ISR_YEAR = 2006  # Easy to update for Setting where Pre VS POST lies.
forecast_start_date = "2024-11-01"
pollutants = ['PM10 Total 0-10um STP', 'PM2.5 - Local Conditions', 'Ozone', 'Nitrogen dioxide (NO2)']
counties = ['San Joaquin', 'Stanislaus', 'Merced', 'Fresno', 'Kings', 'Tulare', 'Kern']

# --- LOAD AND CLEAN ---
df = pd.read_csv("SJV_AQI_1980_2025.csv")
df = df[df['county'].isin(counties) & df['parameter'].isin(pollutants)]
df = df[df['metric_used'] == 'Daily Mean']


# Fix datetime and group
df['datetime'] = pd.to_datetime(df['first_max_datetime'], errors='coerce')
df = df.dropna(subset=['datetime'])
df['date'] = df['datetime'].dt.date
df['month'] = pd.to_datetime(df['datetime'].dt.to_period('M').astype(str))
df['year'] = df['datetime'].dt.year
df = df.rename(columns={'parameter': 'pollutant', 'arithmetic_mean': 'value'})


# --- FUNCTION: Forecast by time unit ---
def forecast_by_timescale(grouped, freq, periods, label):
    results = []
    for (county, pollutant), group in grouped.groupby(['county', 'pollutant']):
        ts = group.set_index('date').asfreq(freq)['value'].fillna(method='ffill')
        if len(ts.dropna()) < 36:
            continue
        try:
            model = SARIMAX(ts, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
            fit = model.fit(disp=False)
            start = len(ts)
            end = start + periods - 1
            forecast_index = pd.date_range(ts.index[-1] + pd.tseries.frequencies.to_offset(freq), periods=periods, freq=freq)
            forecast = fit.predict(start=start, end=end)
            results.append(pd.DataFrame({
                'date': forecast_index,
                'predicted_value': forecast.values,
                'county': county,
                'pollutant': pollutant,
                'scale': label
            }))
        except:
            continue
    return pd.concat(results)



# --- FORECAST EXECUTION ---
daily = df.groupby(['county', 'pollutant', 'datetime'])['value'].mean().reset_index()
daily = daily.rename(columns={'datetime': 'date'})
monthly = df.groupby(['county', 'pollutant', 'month'])['value'].mean().reset_index()
monthly = monthly.rename(columns={'month': 'date'})
yearly = df.groupby(['county', 'pollutant', 'year'])['value'].mean().reset_index()
yearly['date'] = pd.to_datetime(yearly['year'].astype(str) + "-01-01")

daily_forecast = forecast_by_timescale(daily, 'D', 10, 'daily')
monthly_forecast = forecast_by_timescale(monthly, 'MS', 10, 'monthly')
yearly_forecast = forecast_by_timescale(yearly, 'YS', 10, 'yearly')

forecast_df = pd.concat([daily_forecast, monthly_forecast, yearly_forecast])
forecast_df.to_csv("SJV_AQI_Predictions_AllScales.csv", index=False)




# --- PLOT: Per pollutant per timescale ---
for scale in ['daily', 'monthly', 'yearly']:
    scale_df = forecast_df[forecast_df['scale'] == scale]
    for pollutant in scale_df['pollutant'].unique():
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=scale_df[scale_df['pollutant'] == pollutant], x='date', y='predicted_value', hue='county', marker='o')
        plt.title(f"Forecast for {pollutant} ({scale.capitalize()})")
        plt.xticks(rotation=45)
        plt.ylabel("Predicted Value (µg/m³)")
        plt.xlabel("Date")
        plt.tight_layout()
        plt.show()

# --- PLOT: Combined plot per scale, averaged across counties ---
for scale in ['daily', 'monthly', 'yearly']:
    combined = (
        forecast_df[forecast_df['scale'] == scale]
        .groupby(['date', 'pollutant'])['predicted_value']
        .mean()
        .reset_index()
    )
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=combined, x='date', y='predicted_value', hue='pollutant', marker='o')
    plt.title(f"Average Forecast per Pollutant ({scale.capitalize()})")
    plt.xticks(rotation=45)
    plt.ylabel("Average Predicted Value (µg/m³)")
    plt.xlabel("Date")
    plt.tight_layout()
    plt.show()

## 2nd SARIMA Attempt_Jun

In [ ]:
# Load data
merged_df = pd.read_csv("ready_pm25_fresno_with_Date.csv")
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.sort_values('date')
merged_df.set_index('date', inplace=True)

# Target series
series = merged_df['aqi_smoothed'].dropna()

# Train/test split
split = int(len(series) * 0.8)
train, test = series.iloc[:split], series.iloc[split:]

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Assuming train/test are defined
model = SARIMAX(train,
                order=(1,1,1),
                seasonal_order=(1,1,1,7),
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)

# Forecast
forecast = results.forecast(steps=len(test))

# Metrics
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
r2 = r2_score(test, forecast)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

# Plot
plt.figure(figsize=(14,6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test', color='orange')
plt.plot(test.index, forecast, label='Forecast', color='green')
plt.title('SARIMA Forecast vs Actual AQI')
plt.xlabel('Date')
plt.ylabel('AQI (Smoothed)')
plt.legend()
plt.grid(True)
plt.show()


# Statistical Analyses

## T-Test: Pre vs Post ISR

In [ ]:
# --- T-TEST: Pre vs Post ISR ---
ttest_results = []

for county in counties:
    for pollutant in pollutants:
        data = df[(df['county'] == county) & (df['pollutant'] == pollutant)]
        pre = data[data['year'] < ISR_YEAR]['value'].dropna()
        post = data[data['year'] >= ISR_YEAR]['value'].dropna()
        if len(pre) > 10 and len(post) > 10:
            t_stat, p_val = ttest_ind(pre, post, equal_var=False)
            ttest_results.append({
                'county': county,
                'pollutant': pollutant,
                'pre_mean': pre.mean(),
                'post_mean': post.mean(),
                't_stat': t_stat,
                'p_value': p_val,
                'significant': p_val < 0.05
            })

ttest_df = pd.DataFrame(ttest_results)

# Save results to a CSV file
ttest_df.to_csv("ISR_TTest_Results.csv", index=False)

# Print all results (not just significant ones)
print("T-test completed. All results:\n")
print(ttest_df)  # This will print all results, including non-significant ones

 Conclusion: There has been a statistically significant improvement in air quality after 2006.

Why?

Every row shows significant = True, meaning the p-values are all below 0.05 (many even far below 0.001).

For each county and pollutant:

The post_mean is consistently lower than the pre_mean, indicating improved air quality.

The t-statistics are large and positive, which aligns with this drop being statistically meaningful.

📌 What does this mean in plain terms?
For nearly every pollutant and county:

PM10 and PM2.5 levels dropped significantly after 2006, the assumed year of the ISR (Incentive-based Smog Reduction, or similar air regulation).

This suggests that the ISR policy had a real, measurable positive effect on reducing harmful air pollutants in the San Joaquin Valley.

📉 Example Breakdown:

County	Pollutant                        Pre-ISR Avg	  Post-ISR Avg	  Change
Merced	PM2.5 - Local Conditions	         20.86	        13.19	       ↓ 36.8%
Fresno	PM2.5 - Local Conditions	         20.05	        15.05	       ↓ 25.0%
Tulare	PM10 Total 0-10um STP            	 51.14	        44.96	       ↓ 12.1%

All of these are substantial reductions.

🧪 T-test Interpretation Summary:
Metric	Meaning
t_stat	Magnitude of difference relative to variance. Larger = stronger effect.
p_value	Probability result is due to chance. < 0.05 = statistically significant.
significant	Boolean indicating if difference is statistically meaningful.

### T-Test result Visuals

In [ ]:
ttest_df

In [ ]:
# Filter the ttest_df to only include PM2.5 rows
df_pm25_ttest = ttest_df[ttest_df['pollutant'] == 'PM2.5 - Local Conditions']

# Calculate the percentage change and direction for the table
df_pm25_ttest['Change'] = ((df_pm25_ttest['post_mean'] - df_pm25_ttest['pre_mean']) / df_pm25_ttest['pre_mean']) * 100
df_pm25_ttest['Change'] = df_pm25_ttest['Change'].round(1).astype(str) + '%'

# Round Pre-ISR and Post-ISR averages to 2 decimal places
df_pm25_ttest[['pre_mean', 'post_mean']] = df_pm25_ttest[['pre_mean', 'post_mean']].round(2)

# Add direction symbols based on the change value
df_pm25_ttest['Direction'] = df_pm25_ttest['Change'].apply(lambda x: '↓' if '-' in x else '↑')

# Create the final table with selected columns
df_pm25_ttest_table = df_pm25_ttest[['county', 'pollutant', 'pre_mean', 'post_mean', 'Change', 'Direction']]

# Display the table
df_pm25_ttest_table

In [ ]:
# Filter the ttest_df to only include PM2.5 rows
df_pm25_ttest = ttest_df[ttest_df['pollutant'] == 'PM10 Total 0-10um STP']

# Calculate the percentage change and direction for the table
df_pm25_ttest['Change'] = ((df_pm25_ttest['post_mean'] - df_pm25_ttest['pre_mean']) / df_pm25_ttest['pre_mean']) * 100
df_pm25_ttest['Change'] = df_pm25_ttest['Change'].round(1).astype(str) + '%'

# Round Pre-ISR and Post-ISR averages to 2 decimal places
df_pm25_ttest[['pre_mean', 'post_mean']] = df_pm25_ttest[['pre_mean', 'post_mean']].round(2)

# Add direction symbols based on the change value
df_pm25_ttest['Direction'] = df_pm25_ttest['Change'].apply(lambda x: '↓' if '-' in x else '↑')

# Create the final table with selected columns
df_pm25_ttest_table = df_pm25_ttest[['county', 'pollutant', 'pre_mean', 'post_mean', 'Change', 'Direction']]

# Display the table
df_pm25_ttest_table

In [ ]:
# Generate Bar Plot for T-test Results
import matplotlib.pyplot as plt

# Create a figure with subplots for each pollutant
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Barplot for PM10
pm10_data = ttest_df[ttest_df['pollutant'].str.contains("PM10")]
axes[0].bar(pm10_data['county'], pm10_data['pre_mean'], width=0.4, label='Pre-ISR', align='center', color='darkgreen')
axes[0].bar(pm10_data['county'], pm10_data['post_mean'], width=0.4, label='Post-ISR', align='edge', color='lightgreen')
axes[0].set_title('PM10 - Pre vs Post ISR')
axes[0].set_xlabel('County')
axes[0].set_ylabel('AQI')
axes[0].legend()

# Barplot for PM2.5
pm25_data = ttest_df[ttest_df['pollutant'].str.contains("PM2.5")]
axes[1].bar(pm25_data['county'], pm25_data['pre_mean'], width=0.4, label='Pre-ISR', align='center', color='darkgreen')
axes[1].bar(pm25_data['county'], pm25_data['post_mean'], width=0.4, label='Post-ISR', align='edge', color='lightgreen')
axes[1].set_title('PM2.5 - Pre vs Post ISR')
axes[1].set_xlabel('County')
axes[1].set_ylabel('AQI')
axes[1].legend()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()